# Client 동기화

In [316]:
import kfp
import requests

USERNAME = "user@example.com"
PASSWORD = "12341234"
NAMESPACE = "kubeflow-user-example-com"
HOST = "http://127.0.0.1:8080" # istio-ingressgateway pod ip:port

session = requests.Session()
response = session.get(HOST)

headers = {
    "Content-Type": "application/x-www-form-urlencoded",
}

data = {"login": "user@example.com", "password": "12341234"}
session.post(response.url, headers=headers, data=data)
session_cookie = session.cookies.get_dict()["authservice_session"]

client = kfp.Client(
    host=f"{HOST}/pipeline",
    namespace=f"{NAMESPACE}",
    cookies=f"authservice_session={session_cookie}",
)
list_pipelines = client.list_pipelines()

print(list_pipelines.total_size)

#for i in range(list_pipelines.total_size):

25


In [5]:
client

## Pipeline Upload

In [355]:
pipeline_package_path='/Users/gimgihun/file_transfer/practice/test.yaml'
pipeline_name='torch'
client.upload_pipeline(pipeline_package_path=pipeline_package_path,pipeline_name=pipeline_name)

{'created_at': datetime.datetime(2023, 1, 9, 13, 53, 57, tzinfo=tzutc()),
 'default_version': {'code_source_url': None,
                     'created_at': datetime.datetime(2023, 1, 9, 13, 53, 57, tzinfo=tzutc()),
                     'description': None,
                     'id': '717a9a42-bdd0-4aee-868d-b95a663994d8',
                     'name': 'torch',
                     'package_url': None,
                     'parameters': None,
                     'resource_references': [{'key': {'id': '717a9a42-bdd0-4aee-868d-b95a663994d8',
                                                      'type': 'PIPELINE'},
                                              'name': None,
                                              'relationship': 'OWNER'}]},
 'description': None,
 'error': None,
 'id': '717a9a42-bdd0-4aee-868d-b95a663994d8',
 'name': 'torch',
 'parameters': None,
 'resource_references': None,
 'url': None}

# Experiment 생성

In [356]:
experiment = client.create_experiment(name=pipeline_name,description=pipeline_name,namespace=NAMESPACE)

In [357]:
experiment_info = client.get_experiment(experiment_name=pipeline_name,namespace=NAMESPACE)
experiment_id = experiment_info.id
experiment_id


'27dedf49-c92b-4124-b3ee-90745922498d'

## Run 생성

In [358]:
client.run_pipeline(experiment_id=experiment.id,job_name=pipeline_name,pipeline_package_path=pipeline_package_path)

{'created_at': datetime.datetime(2023, 1, 9, 13, 53, 59, tzinfo=tzutc()),
 'description': None,
 'error': None,
 'finished_at': datetime.datetime(1970, 1, 1, 0, 0, tzinfo=tzutc()),
 'id': '198b0ea4-a80c-45e6-a19d-e6610f0dd883',
 'metrics': None,
 'name': 'torch',
 'pipeline_spec': {'parameters': None,
                   'pipeline_id': None,
                   'pipeline_manifest': None,
                   'pipeline_name': None,
                   'runtime_config': None,
                   'workflow_manifest': '{"apiVersion": '
                                        '"argoproj.io/v1alpha1", "kind": '
                                        '"Workflow", "metadata": '
                                        '{"generateName": '
                                        '"practice-pipeline-", "annotations": '
                                        '{"pipelines.kubeflow.org/kfp_sdk_version": '
                                        '"1.8.18", '
                                        '"pipel

In [359]:
pipeline_id = client.get_pipeline_id(pipeline_name)
client.delete_pipeline(pipeline_id=pipeline_id)

{}

In [360]:
experiment_info = client.get_experiment(experiment_name=pipeline_name,namespace=NAMESPACE)
experiment_id = experiment_info.id

In [361]:
client.delete_experiment(experiment_id=experiment_id)

{}

In [362]:
for i in range(client.list_runs().total_size):
    run_id = client.list_runs().runs[i].id
    delete_run_id = run_id
    print(delete_run_id)

client.runs.delete_run(id=delete_run_id)

b66d3771-0ca0-4edb-8e6b-6efb7894e813
41f0e917-0670-4940-830c-2663560c3bfb
be3ff9f3-b609-4439-9363-cef0c5028721
93d5ae8b-3922-41c4-aeb8-6f91dba8bde3
e4372185-883f-4f0f-88fa-b16058a93ad9
4ca37a3e-72a7-45d6-81a7-b65fb3f32838
1d07e319-ab1e-47a9-8673-e4dce9631ff6
faa4817f-5bfa-4943-bb89-6b3323875a54
198b0ea4-a80c-45e6-a19d-e6610f0dd883


{}